# Trigger Examples
Intro to Triggers text 

In [ ]:
# cleanup any old results
!./cleanup.sh

# ascent + conduit imports
import conduit
import conduit.blueprint
import ascent

import numpy as np

# Jupyter imports
from IPython.display import Image
# helpers we use when displaying results in the notebook
from ascent_tutorial_jupyter_utils import img_display_width
from ascent_tutorial_jupyter_utils import ImageSeqViewer
from ascent_tutorial_jupyter_utils import tutorial_gyre_example

import matplotlib.pyplot as plt

In [ ]:
# Use triggers to render when a condition occurs
a = ascent.Ascent()
a.open()

# Setup actions to tell Ascent what to do
# add our expressions definiton to the actions
actions = conduit.Node()

# declare a question to ask 
add_queries = actions.append()
add_queries["action"] = "add_queries"

queries = add_queries["queries"] 
queries["q1/params/expression"] = "entropy(histogram(field('gyre'), num_bins=128))"
queries["q1/params/name"] = "entropy"

# declare triggers 
add_triggers = actions.append()
add_triggers["action"] = "add_triggers"
triggers = add_triggers["triggers"] 

# simple trigger that fires at cycle 500
triggers["t1/params/condition"] = "cycle() == 500"
triggers["t1/params/actions_file"] = "cycle_trigger_actions.yaml"

# a trigger that fires when the change in entroy exceeds 0.5

# the history function allows you to access query results of previous
# cycles. relative_index indicates how far back in history to look.

# Looking at the plot of gyre entropy in the previous notebook, we see a jump
# in entropy at cycle 200, so we expect the trigger to fire at cycle 200
triggers["t2/params/condition"] = "entropy - history(entropy, relative_index = 1) > 0.5"
triggers["t2/params/actions_file"] = "entropy_trigger_actions.yaml"

# view our full actions tree
print(actions.to_yaml())

nsteps = 10
time = 0.0
delta_time = 0.5

info = conduit.Node()
for step in range(nsteps):
    # call helper that generate a double gyre time varying example mesh.
    # gyre ref :https://shaddenlab.berkeley.edu/uploads/LCS-tutorial/examples.html
    mesh = tutorial_gyre_example(time)
    
    # update the examples default cycle
    mesh["state/cycle"] = 100 + step * 100
    time = time + delta_time
    
    # publish mesh to ascent
    a.publish(mesh)
    
    # execute
    a.execute(actions)

# get a copy of the info    
a.info(info)

# close ascent
a.close()

In [ ]:
! ls cycle_trigger*.png

In [ ]:
# show the result image from the cycle trigger
Image("cycle_trigger_out_500.png",width=img_display_width())

In [ ]:
! ls entropy_trigger*.png

In [ ]:
# show the result image from the entropy trigger
Image("entropy_trigger_out_200.png",width=img_display_width())

In [ ]:
print(info["expressions"].to_yaml())